# **Домашнее задание №1 (base)**

В этом домашнем задании вам будет необходимо:
*  обучить модель регрессии для предсказания стоимости автомобилей;
* реализовать веб-сервис для применения построенной модели на новых данных

**Максимальная оценка за дз**
> Оценка за домашку = $min(\text{ваш балл}, 11)$

**Мягкий дедлайн: 27 ноября 23:59**

**Жесткий дедлайн: 20 декабря 23:59 (конец модуля)**


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns

random.seed(42)
np.random.seed(42)

In [61]:
df_train = pd.read_csv('https://raw.githubusercontent.com/Murcha1990/MLDS_ML_2022/main/Hometasks/HT1/cars_train.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/Murcha1990/MLDS_ML_2022/main/Hometasks/HT1/cars_test.csv')

print("Train data shape:", df_train.shape)
print("Test data shape: ", df_test.shape)

Train data shape: (6999, 13)
Test data shape:  (1000, 13)


### Объединим все этапы подготовки в пайплайн, чтобы модель могла делать весь препроцессинг автоматически при подаче дефолтных данных на предсказание.

In [109]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, KFold
from custom_preprocessing import CustomPreprocessing
import pickle
from joblib import dump, load

In [110]:
### колонки для обработки
cols_drop = ['torque', 'name']
cols_get_float = ['mileage', 'engine', 'max_power']
numerical_cols = ['year', 'km_driven', 'mileage', 'engine', 'max_power', 'seats']
categorical_cols = ['fuel', 'seller_type', 'transmission', 'owner']

### преобразователи колонок
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(sparse_output=False, drop='first', handle_unknown='error'))
])

column_trans = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='drop'
)

### Пайплайн с самописным классом препроцессинга
### Приводим поля в нужные форматы, выделяем числовые значения из колонок и тд
pipe = Pipeline(steps=[
    ('preprocessor', CustomPreprocessing(cols_drop=cols_drop, cols_get_float=cols_get_float)),
    ('column_transformer', column_trans),
    ('model', Ridge())
])


In [111]:
%%time
### данные для обучения
dupl_index = df_train.drop(['selling_price'], axis = 1).drop_duplicates(keep='first').index
df_train = df_train[df_train.index.isin(dupl_index)].reset_index(drop = True)
y_train = df_train['selling_price']
X_train = df_train.drop(['selling_price'], axis = 1)

# GridSearchCV - ищем оптимальные параметры
params = {'model__alpha': np.logspace(-10, 2, 100)}
cv = KFold(n_splits=10, random_state=42, shuffle=True)
gs_ridge = GridSearchCV(pipe, params, cv=cv, scoring='r2', verbose=1)
gs_ridge.fit(X_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
CPU times: user 1min 37s, sys: 1min 9s, total: 2min 46s
Wall time: 1min 43s


GridSearchCV(cv=KFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocessor',
                                        CustomPreprocessing(cols_drop=['torque',
                                                                       'name'],
                                                            cols_get_float=['mileage',
                                                                            'engine',
                                                                            'max_power'])),
                                       ('column_transformer',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler...
       1.62975083e-01, 2.15443469e-01, 2.84803587e-01, 3.76493581e-01,
       4.97702356e-01, 6.57933225e-01, 8.69749003e-01, 1.14975700e+00,
       1.51991108e+00, 2.00923300e+00, 2.65608778e+00, 3.51119173e+00,
       4.64158883e+00, 6.13590727e+00, 8.11130831e+00, 1.07226722e+01,
       1.41747416e+01, 1.87381742e+01, 2.47707636e+01, 3.27454916e+01,
       4.32876128e+01, 5.72236766e+01, 7.56463328e+01, 1.00000000e+02])},
             scoring='r2', verbose=1)

In [112]:
###сохраняем модельку
dump(gs_ridge, 'model.pkl')

['model.pkl']

In [113]:
### качество, коэфы модели с лучшим значением параметра
print(f"Качество лучшей модели - R^2 = {gs_ridge.best_score_:.2f}")
print(f"Параметры лучшей модели: {gs_ridge.best_params_}")
print()

coefficients = gs_ridge.best_estimator_.named_steps['model'].coef_
feature_names = gs_ridge.best_estimator_.named_steps['column_transformer'].get_feature_names_out()
coefficients_with_features = list(zip(feature_names, coefficients))

coefficients_with_features = [
    (feature, coef) for feature, coef in coefficients_with_features if isinstance(coef, (int, float))
]
sorted_coefficients = sorted(coefficients_with_features, key=lambda x: abs(x[1]), reverse=True)

print("Feature Coefficients (sorted by magnitude):")
for feature, coef in sorted_coefficients:
    print(f"{feature}: {coef}")

Качество лучшей модели - R^2 = 0.64
Параметры лучшей модели: {'model__alpha': 1e-10}

Feature Coefficients (sorted by magnitude):
cat__owner_Test Drive Car: 3310946.3503775457
cat__transmission_Manual: -303052.97726694634
num__max_power: 271314.81048690225
num__year: 131354.70841720322
cat__seller_type_Trustmark Dealer: -130447.72114707247
cat__fuel_LPG: 118978.01255354838
cat__seller_type_Individual: -104063.50122354022
cat__owner_Second Owner: -60915.53652925441
cat__owner_Third Owner: -50723.877291006786
cat__owner_Fourth & Above Owner: -50004.77591234294
num__engine: 44192.25304636372
num__km_driven: -35678.62821363784
cat__fuel_Petrol: -34641.4239756897
cat__fuel_Diesel: 30439.480023398297
num__mileage: 24246.34701379496
num__seats: -9583.580689304235


# **Часть 5 (3 балла) | Реализация сервиса на FastAPI**

### **Задание 18 (3 балла)**


Cделайте с помощью FastAPI сервис, который с точки зрения пользователя реализует две функции:

1. на вход в формате json подаются признаки одного объекта, на выходе сервис выдает предсказанную стоимость машины
2. на вход подается csv-файл с признаками тестовых объектов, на выходе получаем файл с +1 столбцом - предсказаниями на этих объектах

С точки зрения реализации это означает следующее:
- средствами pydantic должен быть описан класс базового объекта
- класс с коллецией объектов
- метод post, который получает на вход один объект описанного класса
- метод post, который получает на вход коллекцию объектов описанного класса

Шаблон для сервисной части дан ниже. Код необходимо дополнить и оформить в виде отдельного .py-файла.

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List

app = FastAPI()


class Item(BaseModel):
    name: str
    year: int
    selling_price: int
    km_driven: int
    fuel: str
    seller_type: str
    transmission: str
    owner: str
    mileage: str
    engine: str
    max_power: str
    torque: str
    seats: float


class Items(BaseModel):
    objects: List[Item]


@app.post("/predict_item")
def predict_item(item: Item) -> float:
    return ...


@app.post("/predict_items")
def predict_items(items: List[Item]) -> List[float]:
    return ...

In [121]:
### Генерим данные для проверки predict_items_csv
data = [
    {
        "name": "Car A",
        "year": 2015,
        "km_driven": 50000,
        "fuel": "Petrol",
        "seller_type": "Individual",
        "transmission": "Manual",
        "owner": "First Owner",
        "mileage": "23.4 kmpl",
        "engine": "1197 CC",
        "max_power": "83 bhp",
        "torque": "200Nm",
        "seats": 5.0
    },
    {
        "name": "Car B",
        "year": 2017,
        "km_driven": 40000,
        "fuel": "Diesel",
        "seller_type": "Dealer",
        "transmission": "Automatic",
        "owner": "Second Owner",
        "mileage": "20.4 kmpl",
        "engine": "1498 CC",
        "max_power": "110 bhp",
        "torque": "250Nm",
        "seats": 5.0
    },
    {
        "name": "Car C",
        "year": 2018,
        "km_driven": 30000,
        "fuel": "Petrol",
        "seller_type": "Individual",
        "transmission": "Manual",
        "owner": "First Owner",
        "mileage": "21.0 kmpl",
        "engine": "998 CC",
        "max_power": "67 bhp",
        "torque": "150Nm",
        "seats": 4.0
    }
]

df_csv = pd.DataFrame(data)
csv_filename = "predict_items_csv_sample.csv"
df_csv.to_csv(csv_filename, index=False)
print(f"Sample CSV file '{csv_filename}' created successfully.")

Sample CSV file 'predict_items_csv_sample.csv' created successfully.


Протестируйте сервис на корректность работы и приложите скриншоты (см. ниже).

**Сервис реализован с 3 эндпоинтами:**
1. `predict_item` - Прогноз цены продажи авто по входящим признакам
2. `predict_items_list` - Прогноз цен продаж списка авто по входящим признакам
3. `predict_items_csv` - Прогноз цен для нескольких объектов в формате CSV Возвращает CSV файл с +1 колонкой, содержащей прогноз модели

# **Часть 6 (1 балл) | Оформление результатов**

### **Задание 19 (1 балл)**


**Результаты вашей работы** необходимо разместить в своем Гитхабе. Под результатами понимаем следующее:
* ``.ipynb``-ноутбук со всеми проведёнными вами экспериментами (output'ы ячеек, разумеется, сохранить)
* сохраненный дашборд в любом формате
* ``.py``-файл с реализацией сервиса
* ``.pickle``-файл с сохранёнными весами модели, коэффициентами скейлинга и прочими числовыми значениями, которые могут понадобиться для инференса
* ``.md``-файл с выводами про проделанной вами работе:
    * что было сделано
    * с какими результатами
    * что дало наибольший буст в качестве
    * что сделать не вышло и почему (это нормально, даже хорошо😀)

**За что могут быть сняты баллы в этом пункте:**
* за отсутствие ``.pickle``-файла с весами использованной модели
* за недостаточную аналитику в ``.md``-файле
* за оформление и логику кода (в определённом смысле это тоже элемент оформления решения)

**Как будет выглядет проверка всего домашнего задания?**
1. Ассистент проходит по ссылке на (**открытый**) репозиторий из Энитаска
2. Смотрит ``readme.md``:
    * пожалуйста, приложите в него же скрины работы вашего сервиса -- собирать ваши проекты довольно времязатратно, но хочется убедиться, что всё работает
    * можете в md-файл приложить ссылку на screencast с демонстрацией
3. Просматривает ноутбук с DS-частью
4. Заглядывает в код сервиса
5. Хвалит

# **Часть Благодарственная**

Надеемся, вы честно проделали все пункты, а не просто пролистали досюда. Потому что здесь награда за старания. Пожалуйста, не стоит награждать себя до того, как закончите работать над домашкой!

<details>
<summary><b>Что-то приятное</b></summary>

**Напоминаем, что нашем курсе действует система кото-бонусов** 🐈

На фото по ссылке — сэр кот кого-то из команды курса (преподаватель, помощник преподавателя, ассистенты).

Предлагаем вам угадать — чей это товарищ!

[Первый кот](https://ibb.co/XbnpCTg)

</details>



Возможно, это кот Сабрины)